In [115]:
## This is my expanded script from spring 2019

#input: e.g.
# calendar = '201909' #[year][first month of term]
# courses201920W1 = [
#     ('PHYS','216'),
#     ('MATH','211')
# ]
#
# courses = courses201920W1

In [97]:
# 2019/20

# ASTR 	303								Introductory Extragalactic Astronomy
# ASTR	329								Introduction to Observational Astronomy
# MATH 	301								Complex Variables
# MATH	342								Intermediate Ordinary Differential Equations
# MATH	346								Introduction to Partial Differential Equations
# PHYS	317								Thermodynamics
# PHYS 	321A							Classical Mechanics I (vital)
# PHYS	321B							Classical Mechanics II
# PHYS	323								Quantum Mechanics I (vital)
# PHYS	325								Optics
# PHYS	326								Electricity and Magnetism (vital)
# E STAT 	260								Math breadth elective

courses201920W1 = [
    ('PHYS','326'),
    ('PHYS','323'),
    ('PHYS','321A'),
    ('ASTR','329'),
    ('MATH','342')
]
# 18 or 23: check A329

# ('MATH','342') is fall or spring

courses201920W2 = [
    ('PHYS','325'),
    ('PHYS','321B'),
    ('PHYS','317'),
    ('MATH','346'),
    ('MATH','301'),
    ('ASTR','303')
]

In [124]:
## SET THIS
calendar = '201909'
courses = courses201920W1
##

In [125]:
def linkBuilder(subject, course, calendar):
    '''Returns the link for the selected course.
    subject:'PHYS','MATH','ASTR', etc.
    course:'216', etc.
    calendar='201909', etc.
    '''
    l = 'https://www.uvic.ca/BAN1P/bwckctlg.p_disp_listcrse?'
    l += 'term_in=' + calendar
    l += '&subj_in=' + subject
    l += '&crse_in=' + course
    l += '&schd_in='
    return (l, subject, course)

In [126]:
def getCourseInfoFromLink(link, subject, course):
    '''Returns a list of course info: crn, section, time(start,end), days
    '''
    from lxml import html
    import requests
    import math
    tree = html.fromstring(requests.get(link).content)
    
    crns = []; secs = []
    for e in tree.xpath('//a[contains(@href, "crn_in")]/text()'):
        s = e.split('-')
        crns.append(s[1].strip()) #crn
        secs.append(s[3].strip()) #section
        
    hrs = []; days = []
    dat = tree.xpath('//table[@class="datadisplaytable"]/tr/td/text()')
    for i,x in enumerate(dat):
        x = x.strip()
        if x == "Every Week":
            hrs.append(dat[i+1].split(' - ')) #time: start,end
            days.append(dat[i+2]) #days
     
    subs = [subject for i in range(0,len(hrs))]
    crss = [course for i in range(0,len(hrs))]
    
    l = []
    for i in range(0,len(hrs)):
        j = math.ceil((i + 1)/(len(hrs)/len(crns))) - 1
        l.append([crns[j], subject, course, secs[j], hrs[i], days[i]])
    
    return l

In [127]:
def convertCourseInfoForScheduler(inf):
    '''Return inf formatted for Course Schedule Options script
    '''
    crsConvs = {'ASTR':'A','MATH':'M', 'PHYS':'P'}
    secConvs = {'A':'L', 'B':'B', 'T':'T'}
    def tConv(t):    
        m = t[-2:] #am/pm
        tA = int(t[0:t.find(':')]) #hours
        tB = int(t[t.find(':')+1:-3]) #minutes
        if m == 'am' and tA == 12: tA = 0 #to 24 hour
        if m == 'pm' and tA != 12: tA += 12
        if tB < 15: tB = 0 #minutes to decimal
        elif tB < 45: tB = 0.5
        else: tB = 0; tA += 1
        if tA >= 24: tA -= 24 #fix overun from minutes
        return tA+tB
    
    l = []
    for item in inf:
        crn, sub, crs, sec, times, days = item
        tbeg = times[0]; tend = times[1]
        A = crsConvs[sub]+crs+secConvs[sec[0]]
        B = int(sec[1:3])
        C = days
        D = tConv(tbeg)
        E = tConv(tend)
        F = int(crn)
        l.append([A,B,C,D,E,F])
        
    return l

In [128]:
## Scrape data
data = []
for dpmt, crs in courses:
    data += convertCourseInfoForScheduler(getCourseInfoFromLink(*linkBuilder(dpmt, crs, calendar)))

In [129]:
## Here starts my (mostly) original script from spring 2018

#   course, section, days, times
#   eg: P215L, 1, TWF, 9:30-10:20
#       A250B, 2, TR, 15:30-16:50
#   course:
#       P*=physics, A*=astronomy, M*=math
#       *L=lecture, *B=lab, *T=tutorial
#       Note that lectures are L rather than A
#   days:
#       M=monday, T=tuesday, W=wednesday, R=thursday, F=friday
#       Note that thursdays are R rather than Th

In [130]:
#Section indexes
i_course = 0; i_section = 1; i_days = 2; i_stime = 3; i_etime = 4; i_ident = 5

In [131]:
def organize_datum(datum, org_data):
    for group_i in range(len(org_data)):
        if org_data[group_i][0][i_course] == datum[i_course]\
        and org_data[group_i][0][i_ident] != datum[i_ident]:
            org_data[group_i].append(datum)
            return org_data
    org_data.append([datum])
    return org_data
org_data = []
for datum in data:
    org_data = organize_datum(datum, org_data)

In [132]:
#Function to check for day/time conflicts between datum and the data in working_schedule
def conflict(new_section, schedule):
    for existing_section in schedule:
        if set(new_section[i_days]).intersection(set(existing_section[i_days])): #common days
            #check if start or end time of either section is within the other
            if existing_section[i_stime] <= new_section[i_stime] < existing_section[i_etime]: return True
            if existing_section[i_stime] < new_section[i_etime] <= existing_section[i_etime]: return True
            if new_section[i_stime] <= existing_section[i_stime] < new_section[i_etime]: return True
            if new_section[i_stime] < existing_section[i_etime] <= new_section[i_etime]: return True
    return False

In [133]:
#Function to determine possible schedule options from data
def determine_schedules(_pool, _schedule, schedules):
    pool = list(_pool)
    sections = pool.pop(0)
    for section in sections:
        schedule = list(_schedule)
        if not conflict(section, schedule):
            schedule.append(section)
            if pool:
                schedules = determine_schedules(pool, schedule, schedules)
            else:
                schedules.append(schedule)
    return schedules

In [134]:
#Headers
def printHeader(dat):
    print("Using data:")
    print("A=ASTR, P=PHYS, M=MATH, L=lecture, B=lab, T=tutorial, days=MTWRF") #legend
    print("Course  Section  Start  End  ID     Days")
    for crs, sec, days, tbeg, tend, crn in dat:
        print("%6s %8d %6.1f %4.1f %6d %5s" % (crs, sec, tbeg, tend, crn, days))

printHeader(data)

#All possible schedules without conflicts
schedules = determine_schedules(org_data, [], [])

#Output schedules
tracker = 1
for schedule in schedules:
    print("%02d ________________________________________________________" % (tracker))
    print("|       |    M    |    T    |    W    |    R    |    F    |")

#     ident_tracker = 0
    for t in range(16,45,1):
        time = t/2.

        #time
        print("| %02d:%s |" % (int(time), "00" if time.is_integer() else "30"), end='')

        #courses at this time each day
        for day in ['M','T','W','R','F']:
            for datum in schedule:
                if day in datum[i_days] and datum[i_stime] <= time < datum[i_etime]:
                    print("%6s%02d |" % (datum[i_course], datum[i_section]), end='')
                    break
            else:   
                print("%10s" % "|", end='')

#         #ident's
#         if not ident_tracker >= len(schedule):
#             print(schedule[ident_tracker][i_ident], end='')
#             ident_tracker += 1

        print("") #go to next line

    print("-----------------------------------------------------------")
    
    #tab separated crn's
    print(*set([s[i_ident] for s in schedule]), sep='\t')
    
    print("") #space between schedules
 
    tracker += 1

print("Total possible schedule options: %d" % (len(schedules)))

Using data:
A=ASTR, P=PHYS, M=MATH, L=lecture, B=lab, T=tutorial, days=MTWRF
Course  Section  Start  End  ID     Days
 P326L        1    8.5  9.5  12608   TWF
 P326B        1   14.5 17.5  12609     M
 P326B        2   15.5 18.5  12610     W
 P326T        1   12.5 13.5  12611     W
 P323L        1   10.0 11.5  12601    MR
 P323B        1   14.5 17.5  12602     M
 P323B        2   16.5 19.5  12603     T
 P323B        3   14.5 17.5  12604     W
 P323B        4   14.5 17.5  12605     R
 P323B        5   14.5 17.5  12606     F
 P323T        1   12.5 13.5  12607     T
P321AL        1   10.5 11.5  12594   TWF
P321AB        1   14.5 17.5  12595     M
P321AB        2   16.5 19.5  12596     T
P321AB        3   14.5 17.5  12597     W
P321AB        4   14.5 17.5  12598     R
P321AB        5   14.5 17.5  12599     F
P321AT        1   12.5 13.5  12600     F
 A329B        1   13.0 14.5  10226    MR
 A329B        1   14.5 17.5  10226     M
 M342L        1   11.5 12.5  12187   TWF
01 __________________